In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import numpy as np
import tqdm
import random
import torch.nn.functional as F

# for tensorboard
from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter()

# Using embedding
In this experiment, we sample data not only from uniform distribution but from variety of distributions. We then use this data for encoder and feed its output throught cross-attention mechanism to the decoder. The decoder takes the sequence with missing tokens and tries to fill in the gaps.

### Architecture
- We use multihead attention for the encoder and vanilla attention for the decoder.
- In the encoder, before splitting heads I "merge" the 5 sequences through linear layer.

In [2]:
cd semester-project/

[Errno 2] No such file or directory: 'semester-project/'
/Users/mariayuffa/semester-project


In [55]:
# Sequences for decoder
final_chains_train = np.load('data/final_chains_T=1_num_iters=400_train.npy')
chains_train = final_chains_train.copy()
chains_train[final_chains_train==-1] = 1
chains_train[final_chains_train==1] = 3
tensor_chains_train = torch.tensor(chains_train, dtype=torch.long)
print("Loaded train sequences of proteins sampled from Boltzmann distribution:", final_chains_train.shape)

final_chains_test = np.load('data/final_chains_T=1_num_iters=400_test.npy')
chains_test = final_chains_test.copy()
chains_test[final_chains_test==-1] = 1 
chains_test[final_chains_test==1] = 3 
tensor_chains_test = torch.tensor(chains_test, dtype=torch.long)
print("Loaded test sequences of proteins sampled from Boltzmann distribution:", final_chains_test.shape)


# Sequences for encoder
k = 0
final_chains_encoder_train = np.zeros((1000,5,200))
final_chains_encoder_train[:,0,:] = np.load('data/final_chains_T=1_num_iters=400_J=10_test.npy') 
final_chains_encoder_test = np.zeros((1000,5,200))
final_chains_encoder_test[:,0,:] = np.load('data/final_chains_T=1_num_iters=400_J=10_test.npy') 
for i in range(20,100,20):
    k+=1
    final_chains_encoder_train[:,k,:] = np.load('data/final_chains_T=1_num_iters=400_J='+str(i)+'_train.npy')
    final_chains_encoder_test[:,k,:] = np.load('data/final_chains_T=1_num_iters=400_J='+str(i)+'_test.npy') 

print("Loaded train sequences of proteins for encoder distribution:", final_chains_encoder_train.shape)
print("Loaded test sequences of proteins for encoder distribution:", final_chains_encoder_test.shape)

samples_train = final_chains_encoder_train.copy() 
samples_train[final_chains_encoder_train==-1] = 1
samples_train[final_chains_encoder_train==1] = 3
tensor_samples_train = torch.tensor(samples_train, dtype=torch.long)

samples_test = final_chains_encoder_test.copy()
samples_test[final_chains_encoder_test==-1] = 1
samples_test[final_chains_encoder_test==1] = 3
tensor_samples_test = torch.tensor(samples_test, dtype=torch.long)

Loaded train sequences of proteins sampled from Boltzmann distribution: (1000, 200)
Loaded test sequences of proteins sampled from Boltzmann distribution: (1000, 200)
Loaded train sequences of proteins for encoder distribution: (1000, 5, 200)
Loaded test sequences of proteins for encoder distribution: (1000, 5, 200)


In [56]:
# Define the parameters for different distributions
distributions = [{"type": "normal", "mean": 0, "std": 1},
        {"type": "uniform", "low": -1, "high": 1},
        {"type": "exponential", "scale": 1},
        {"type": "gamma", "scale": 1},
        {"type": "poisson", "lam": 1.0}]

def sample_data(num_samples, distributions, len_of_seq):
    # Define the parameters for different distributions
    
    # Initialize the tensor to store the samples
    all_samples = torch.zeros(len(distributions), num_samples, len_of_seq)
    
    # Sample data from each distribution
    for i in range(len(distributions)):
        distr = distributions[i]
        if distr["type"] == "normal":
            samples = np.random.normal(distr["mean"], distr["std"], (num_samples, len_of_seq))
            samples = np.where(samples >= 0, 1, -1)
        elif distr["type"] == "uniform":
            samples = np.random.uniform(distr["low"], distr["high"], (num_samples, len_of_seq))
            samples = np.where(samples >= 0, 1, -1)
        elif distr["type"] == "exponential":
            samples = np.random.exponential(distr["scale"], (num_samples, len_of_seq))
            samples = np.where(samples >= distr["scale"], 1, -1)
        elif distr["type"] == "gamma":
            samples = np.random.poisson(distr["scale"], (num_samples, len_of_seq))
            samples = np.where(samples >= np.mean(samples), 1, -1)
        elif distr["type"] == "poisson":
            samples = np.random.poisson(distr["lam"], (num_samples, len_of_seq))
            samples = np.where(samples >= np.mean(samples), 1, -1)
        else:
            raise ValueError("Unsupported distribution type")
        
        # Store the samples in the tensor
        all_samples[i] = torch.tensor(samples, dtype=torch.float32)
    
    return all_samples.reshape(num_samples, len(distributions), len_of_seq)

# Example usage
#tensor_samples_train = sample_data(1000, distributions, 200)  # 5 distributions, 1000 samples each, length of sequence 10
#print(tensor_samples_train.shape)

#tensor_samples_test = sample_data(1000, distributions, 200)  # 5 distributions, 1000 samples each, length of sequence 10
#print(tensor_samples_test.shape)

In [73]:
#data_train = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(1000)])
#data_train_dec = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(1000)])
#data_test = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(400)])
#data_test_dec = torch.tensor([np.random.choice([-1, 1], size=20) for _ in range(400)])
def mask_random_spins(sequence, vocab, pos=25, mask_token=2):
    """
    Mask one random spin in a sequence of protein spins.
    
    Parameters:
    - sequence: a list or sequence of spins (integers)
    - mask_token: the token used to mask a spin (default is 2)
    
    Returns:
    - masked_sequence: a sequence similar to the input but with one spin masked
    - masked_position: the position of the spin that was masked
    """
    # Ensure the sequence can be converted to a list for masking
    sequence_list = sequence.numpy().tolist() if isinstance(sequence, torch.Tensor) else list(sequence)
    # Choose a random position to mask, excluding the first spin
    mask_positions = random.sample(range(len(sequence)), pos)
    
    # Mask the chosen position
    masked_sequence = sequence_list.copy()
    for mask_position in mask_positions:
        masked_sequence[mask_position] = mask_token

    # Create an array of zeros with shape (len(sequence), len(vocab))
    #one_hot = one_hot_encoding(masked_sequence, vocab)
        
    return torch.tensor(masked_sequence, dtype=torch.long), torch.tensor(mask_positions, dtype=torch.long)

In [74]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert embed_dim % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        self.W_q = nn.Linear(embed_dim, embed_dim) # Linear layer for query 
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
        self.W_o = nn.Linear(embed_dim, embed_dim) # Output layer to ensure that dimensionality matches the model's expected dimensionality
        
        self.combine_heads = nn.Linear(num_heads, 1)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, embed_dim = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
     
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        #print("x somewhere:", x.shape)
        #print("seqeunce length:",seq_length)
        #print("d model dim:",self.d_model)
        return x.transpose(1, 2).contiguous().view(batch_size,seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        #print("shape of Q before splitting heads:", Q.shape)
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        #print("shape of Q after splitting heads:", Q.shape)
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        #print(" before W_o:", self.combine_heads(attn_output.permute(1,2,0)).squeeze(-1).shape)
        output = self.W_o(self.combine_heads(attn_output))
        return output
    
class VanillaAttention(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins=3, dropout_rate=0.0):
        super(VanillaAttention, self).__init__()
        self.word_embeddings = nn.Linear(num_spins, embed_dim)
        self.position_embeddings = nn.Embedding(max_seq_length, embed_dim)
        self.a = a  # parameter controlling how important positions are
        self.value_weight = nn.Linear(embed_dim, embed_dim)
        self.query_weight = nn.Linear(embed_dim, embed_dim)
        self.key_weight = nn.Linear(embed_dim, embed_dim)
        self.fc = nn.Linear(embed_dim, embed_dim)  # output layer
        self.dropout = nn.Dropout(dropout_rate)
        self.scale = math.sqrt(embed_dim)

    def forward(self, s, enc_output):

        position_ids = torch.arange(s.size(0), dtype=torch.long)
        #x = self.word_embeddings(s) + self.a*self.position_embeddings(position_ids)
        x = s + self.a*self.position_embeddings(position_ids)
        
        query = self.query_weight(x)
        key = self.key_weight(enc_output)
        values = self.value_weight(enc_output)
        
        # Simple attention score calculation (Dot product): this is equivalent to the interaction matrix
        scores = torch.matmul(query, key.transpose(-2, -1)) / self.scale # Transpose last two dimensions for matrix multiplication
        attn_weights = torch.softmax(scores, dim=-1)  # Apply softmax to scores to get probabilities

        # Apply attention scores to values
        attn_output= torch.matmul(attn_weights, values)

        # Sum over the sequence length dimensions
        #attn_output = attn_output.sum(dim=1)
        output = self.fc(self.dropout(attn_output)) # should have size (20,3)

        return output, attn_weights

In [75]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [76]:
class EncoderLayer(nn.Module):
    def __init__(self, embed_dim, proj_layer_dim, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(embed_dim=embed_dim, num_heads=len(distributions))
        self.feed_forward = PositionWiseFeedForward(d_model=embed_dim, d_ff=proj_layer_dim)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(distributions),1)
        
    def forward(self, x):
        x = self.fc(x.permute(1,2,0)).permute(2,0,1)
        attn_output = self.self_attn(x, x, x)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
    
class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = VanillaAttention(embed_dim=embed_dim, a=a, max_seq_length=max_seq_length, num_spins=num_spins, dropout_rate=dropout)# masking one of the word in the sequence
        self.cross_attn = VanillaAttention(embed_dim=embed_dim, a=a, max_seq_length=max_seq_length, num_spins=num_spins, dropout_rate=dropout)
        self.feed_forward = PositionWiseFeedForward(d_model=embed_dim, d_ff=proj_layer_dim)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.norm3 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

        self.cross_attn_weights = None 
        
    def forward(self, x, enc_output):
        enc_output = enc_output.squeeze(0)
        attn_output,__ = self.self_attn(x, x)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output, cross_attn_weights = self.cross_attn(x, enc_output)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        #x = x.sum(dim=0)

        self.cross_attn_weights = cross_attn_weights 
        return x

In [77]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        print(x.shape)
        print(self.pe.shape)
        return x + self.pe[:, :x.size(1)]

In [92]:
class Transformer(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(num_spins+1, embed_dim)
        self.decoder_embedding = nn.Embedding(num_spins+1, embed_dim)

        #self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        #self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.encoder_layer = EncoderLayer(embed_dim, proj_layer_dim, dropout)
        self.decoder_layer = DecoderLayer(embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout)
        self.fc = nn.Linear(embed_dim, num_spins)

    def forward(self, src, tgt):
        #src_mask, tgt_mask = self.generate_mask(src, tgt)
        #src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        #tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        src_embedded = self.encoder_embedding(src)
        tgt_embedded = self.decoder_embedding(tgt)
        enc_output = self.encoder_layer(src_embedded)
        dec_output = self.decoder_layer(tgt_embedded, enc_output)
        output = self.fc(dec_output)
        #print("output of the transformer:", output.shape)
        return output
    
    def get_cross_attention_weights(self):
        return self.decoder_layer.cross_attn_weights

## Training and validation

In [93]:
writer = SummaryWriter('runs/embedding_run_1_(1,3)')

def evaluate(model, data_test, data_test_dec, vocab, criterion, device=0):
    model.eval()
    epoch_loss = 0
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    for i, data in tqdm.tqdm(enumerate(data_test), total=len(data_test)):
        # Get the inputs
        input_seq_enc = data
        input_seq_dec = data_test_dec[i]
        
        #input_encoder_one_hot = torch.stack([one_hot_encoding(input_seq_enc[i].tolist(), vocab) for i in range(len(input_seq_enc))], dim=0)

        #input_decoder_one_hot = one_hot_encoding(input_seq_dec.tolist(), vocab)
        # mask a token
        masked_sequence_dec, positions = mask_random_spins(input_seq_dec, vocab, mask_token=2)
        
        # Forward pass
        outputs = model.forward(input_seq_enc, masked_sequence_dec)

        #output_token = F.log_softmax(outputs, dim=-1)
        target_tokens = input_seq_dec[positions] #target_token = input_seq[position]
        target_tokens = torch.tensor([vocab[int(token)] for token in target_tokens], dtype=torch.long)

        # Compute loss
        loss = criterion(outputs[positions], target_tokens)
        epoch_loss += loss.item()
    return epoch_loss / len(data_test)

def train(model, data_train, data_train_dec, data_test, data_test_dec, vocab, optimizer, criterion, num_epochs=15, device=0):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    # Training loop
    model.train()
    best_eval_loss = 1e-3 # used to do early stopping

    for epoch in tqdm.tqdm(range(num_epochs), leave=False, position=0):
        running_loss = 0
        epoch_loss = 0
        
        for i, data in tqdm.tqdm(enumerate(data_train), total=len(data_train)):
            # Get the inputs
            input_seq_enc = data
            input_seq_dec = data_train_dec[i]

            input_seq_enc = torch.tensor(input_seq_enc, dtype=torch.long)

            # mask a token in decoder
            masked_sequence_dec, positions = mask_random_spins(input_seq_dec, vocab, mask_token=2)

            # Forward pass
            predictions = model.forward(input_seq_enc, masked_sequence_dec) #masked_sequence[masked_position]
            
            #predicted_token = F.log_softmax(prediction, dim=-1)
            target_token = input_seq_dec[positions] #input_seq[masked_position]
            target_token = torch.tensor([vocab[int(token)] for token in target_token], dtype=torch.long)
            
            # Compute loss
            #print("model prediction:", predictions[positions])
            #print("target:", target_token)
            loss = criterion(predictions[positions], target_token)
            epoch_loss += loss.item()
            
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 10 == 9. :    # print every 10 mini-batches
                writer.add_scalar("Running Loss", running_loss / 100, epoch)
                #print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

        print(f'Epoch {epoch + 1} | Train Loss: {(epoch_loss / len(data)):.4f}')
        writer.add_scalar("Train Loss", epoch_loss / len(data), epoch)
        eval_loss = evaluate(model, data_test, data_test_dec, vocab, criterion, device=device)
        writer.add_scalar("Eval Loss", eval_loss, epoch)
        print(f'Epoch {epoch + 1} | Eval Loss: {(eval_loss):.4f}')
        
        # Perform early stopping based on eval loss
        if eval_loss < best_eval_loss:
            return epoch_loss / len(data_train)
    return epoch_loss / len(data_train)

writer.flush()
writer.close()

In [64]:
# Define the parameters 
vocab_size = 3
vocab = {1:0,3:1,2:2} 
L = 200
embedding_dim = 200
proj_layer_dim = 128
hidden_dim = 200
num_layers = 1 # have to adapt the model for 2 and 3 layers
dropout_rate = 0.0
lr = 1e-3
num_sequences = 1000
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [65]:
# Example usage:

model = Transformer(embed_dim=embedding_dim, a=0, max_seq_length=L, num_spins=3, proj_layer_dim=128, dropout=dropout_rate)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train(model, tensor_samples_train, tensor_chains_train, tensor_samples_test, tensor_chains_test, vocab, optimizer, criterion, device=device)
#torch.save(model.state_dict(), 'models/lstm_scratch.pt')
#evaluate(model, test_dataloader, criterion, device=device)

  0%|          | 0/15 [00:00<?, ?it/s]

/var/folders/mg/44g5ch495hdc8x4_5mq3dsdm0000gn/T/ipykernel_67461/738537234.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_seq_enc = torch.tensor(input_seq_enc, dtype=torch.long)
100%|██████████| 1000/1000 [00:11<00:00, 83.88it/s]


Epoch 1 | Train Loss: 143.4672


  7%|▋         | 1/15 [00:15<03:37, 15.52s/it]

Epoch 1 | Eval Loss: 0.6973


...

100%|██████████| 1000/1000 [00:10<00:00, 91.33it/s]


Epoch 15 | Train Loss: 138.7501


100%|██████████| 1000/1000 [00:03<00:00, 255.88it/s]
                                               

Epoch 15 | Eval Loss: 0.6926


0.6937502991557122

In [66]:
cross_attention_weights = model.get_cross_attention_weights()
print("Cross-attention weights:", cross_attention_weights)

Cross-attention weights: tensor([[5.1106e-26, 5.9657e-25, 5.9657e-25,  ..., 5.9657e-25, 4.7772e-25,
         5.9657e-25],
        [1.6042e-26, 1.9441e-25, 1.9441e-25,  ..., 1.9441e-25, 1.5512e-25,
         1.9441e-25],
        [1.6042e-26, 1.9441e-25, 1.9441e-25,  ..., 1.9441e-25, 1.5512e-25,
         1.9441e-25],
        ...,
        [1.5002e-25, 1.6931e-24, 1.6931e-24,  ..., 1.6931e-24, 1.3603e-24,
         1.6931e-24],
        [1.5002e-25, 1.6931e-24, 1.6931e-24,  ..., 1.6931e-24, 1.3603e-24,
         1.6931e-24],
        [1.6042e-26, 1.9441e-25, 1.9441e-25,  ..., 1.9441e-25, 1.5512e-25,
         1.9441e-25]], grad_fn=<SoftmaxBackward0>)


In [67]:
# To save only the decoder layer weights
torch.save(model.decoder_layer.state_dict(), 'model_decoder/decoder_weights.pth')
torch.save(model.get_cross_attention_weights(), 'model_decoder/transformer_cross_attn_weights.pth')

# If you need to load these weights later
decoder_weights = torch.load('model_decoder/decoder_weights.pth')
model.decoder_layer.load_state_dict(decoder_weights)

# Save the weights of the FC layer
torch.save(model.fc.state_dict(), 'model_decoder/transformer_fc_weights.pth')

# To load these weights back into the FC layer later
fc_weights = torch.load('model_decoder/transformer_fc_weights.pth')
model.fc.load_state_dict(fc_weights)

<All keys matched successfully>

# Ablation study
In this study we remove the encoder when testing the model

In [111]:
class TransformerAblated(nn.Module):
    def __init__(self, embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout):
        super(TransformerAblated, self).__init__()
        self.word_embeddings = nn.Embedding(num_spins+1, embed_dim)
        self.decoder_layer = DecoderLayer(embed_dim, a, max_seq_length, num_spins, proj_layer_dim, dropout)
        self.fc = nn.Linear(embed_dim, num_spins)

    def forward(self, tgt):
        tgt_embedded = self.word_embeddings(tgt)
        dec_output = self.decoder_layer(tgt_embedded, tgt_embedded)
        output = self.fc(dec_output)
        return output

In [112]:
# Create an instance of the new model
new_model = TransformerAblated(embed_dim=embedding_dim, a=0, max_seq_length=L, num_spins=3, proj_layer_dim=128, dropout=dropout_rate)

# Load the saved decoder weights
decoder_weights = torch.load('model_decoder/decoder_weights.pth')
new_model.decoder_layer.load_state_dict(decoder_weights)

# Load the saved FC weights
fc_weights = torch.load('model_decoder/transformer_fc_weights.pth')
new_model.fc.load_state_dict(fc_weights)


<All keys matched successfully>

In [119]:
writer = SummaryWriter('runs/transformer_ablation_encoding_run_2')

def evaluate(new_model, data_test, data_test_dec, vocab, criterion, device=0):
    new_model.eval()
    model.eval()
    epoch_loss = 0
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    for i, data in tqdm.tqdm(enumerate(data_test), total=len(data_test)):
        # Get the inputs
        input_seq_dec = data_test_dec[i]
        # mask a token
        masked_sequence_dec, positions = mask_random_spins(input_seq_dec, vocab, mask_token=2)
        
        # Forward pass
        outputs = new_model.forward(masked_sequence_dec)

        #output_token = F.log_softmax(outputs, dim=-1)
        target_tokens = input_seq_dec[positions] #input_seq[masked_position]
        target_tokens = torch.tensor([vocab[int(token)] for token in target_tokens], dtype=torch.long)

        # Compute loss
        loss = criterion(outputs[positions], target_tokens)
        epoch_loss += loss.item()

    return epoch_loss / len(data_test)

def train(model, new_model, data_train, data_train_dec, data_test, data_test_dec, vocab, optimizer, criterion, num_epochs=20, device=0):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    # Training loop
    model.train()
    best_eval_loss = 1e-3 # used to do early stopping

    for epoch in tqdm.tqdm(range(num_epochs), leave=False, position=0):
        running_loss = 0
        epoch_loss = 0
        
        for i, data in tqdm.tqdm(enumerate(data_train), total=len(data_train)):            
            # Get the inputs
            input_seq_enc = torch.tensor(data, dtype=torch.long)
            input_seq_dec = data_train_dec[i]

            # mask a token in decoder
            masked_sequence_dec, positions = mask_random_spins(input_seq_dec, vocab, mask_token=2)

            # Forward pass
            predictions = model.forward(input_seq_enc, masked_sequence_dec) #masked_sequence[masked_position]
            
            #predicted_token = F.log_softmax(prediction, dim=-1)
            target_tokens = input_seq_dec[positions] #input_seq[masked_position]
            target_tokens = torch.tensor([vocab[int(token)] for token in target_tokens], dtype=torch.long)
            
            # Compute loss
            #print("model prediction:", prediction.shape)
            #print("target:", target_token.shape)
            loss = criterion(predictions[positions], target_tokens)
            epoch_loss += loss.item()
            
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 10 == 9. :    # print every 10 mini-batches
                writer.add_scalar("Running Loss", running_loss / 100, epoch)
                #print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

        new_model.decoder_layer = model.decoder_layer
        new_model.fc = model.fc
        print(f'Epoch {epoch + 1} | Train Loss: {(epoch_loss / len(data)):.4f}')
        writer.add_scalar("Train Loss", epoch_loss / len(data), epoch)
        eval_loss = evaluate(new_model, data_test, data_test_dec, vocab, criterion, device=device)
        writer.add_scalar("Eval Loss", eval_loss, epoch)
        print(f'Epoch {epoch + 1} | Eval Loss: {(eval_loss):.4f}')
        
        # Perform early stopping based on eval loss
        if eval_loss < best_eval_loss:
            return epoch_loss / len(data_train)
    return epoch_loss / len(data_train)

writer.flush()
writer.close()

In [120]:
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
vocab = {1:0,3:1,2:2}

# Define the parameters 
L = 200
embedding_dim = 200
proj_layer_dim = 128
hidden_dim = 200
num_layers = 1 # have to adapt the model for 2 and 3 layers
dropout_rate = 0.0
lr = 1e-2
num_sequences = 1000
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

train(model, new_model, tensor_samples_train, tensor_chains_train, tensor_samples_test, tensor_chains_test, vocab, optimizer, criterion, device=device)


mps


  0%|          | 0/20 [00:00<?, ?it/s]/var/folders/mg/44g5ch495hdc8x4_5mq3dsdm0000gn/T/ipykernel_67461/3935437701.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_seq_enc = torch.tensor(data, dtype=torch.long)
100%|██████████| 1000/1000 [00:10<00:00, 94.09it/s]


Epoch 1 | Train Loss: 138.7359


  5%|▌         | 1/20 [00:12<03:49, 12.08s/it]

Epoch 1 | Eval Loss: 0.6925


...

100%|██████████| 1000/1000 [00:09<00:00, 102.46it/s]


Epoch 20 | Train Loss: 138.7364


100%|██████████| 1000/1000 [00:02<00:00, 462.36it/s]
                                               

Epoch 20 | Eval Loss: 0.6924


0.6936820223927498

# Extracting activations

In [ ]:
# loading self-attention weights
cross_attention_weights = torch.load('model_decoder/transformer_cross_attn_weights.pth')
#model.decoder_layer.load_state_dict(cross_attention_weights)

### need to think what to do with them